intro

goals
summary

acquire data from ...

In [7]:
import acquireTitanic

df = acquireTitanic.get_data()

Prep data by...

- encode embarked
- drop embarked and embark_town (interdependent on each other and represented in 

In [8]:
import pandas as pd

print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passenger_id  891 non-null    int64  
 1   survived      891 non-null    int64  
 2   pclass        891 non-null    int64  
 3   sex           891 non-null    object 
 4   age           714 non-null    float64
 5   sibsp         891 non-null    int64  
 6   parch         891 non-null    int64  
 7   fare          891 non-null    float64
 8   embarked      889 non-null    object 
 9   class         891 non-null    object 
 10  deck          203 non-null    object 
 11  embark_town   889 non-null    object 
 12  alone         891 non-null    int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB
None


,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,None,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,None,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,None,Southampton,1


**to-do**

1. Class/pclass: Do pclass and class represent the same info? If so, drop class and keep pclass because it is represented as an integer.
1. Dummy Vars: Are embarked and embark_town represent the same information? If so, get dummy variables for sex, embarked
1. Drop columns: embarked and sex columns (now dummy vars), deck (almost 700 missing values), class (#1 above was a YES). 
1. Drop rows: where age is missing (177 rows are missing values, and age will likely be a very influential feature). 
1. Write prep function
1. Split data
1. Write split function
1. Write prep_split_function

*1. Class/pclass: Do pclass and class represent the same info? If so, drop class and keep pclass because it is represented as an integer.*

In [9]:
pd.crosstab(df['class'], df['pclass'])

pclass,1,2,3
class,,,
First,216,0,0
Second,0,184,0
Third,0,0,491


Yes! Add 'class' to the list of columns to drop. 

*2. Dummy Vars: Do embarked and embark_town represent the same information? If so, get dummy variables for sex, embarked*

In [10]:
pd.crosstab(df.embarked, df.embark_town)

embark_town,Cherbourg,Queenstown,Southampton
embarked,,,
C,168,0,0
Q,0,77,0
S,0,0,644


YES! Get dummy variables for sex, embarked

In [11]:
# get the boolean variables for classes in the sex and embarked columns, dropping one of the categories for each 
# original column

dummies_df = pd.get_dummies(df[['sex', 'embarked']], drop_first=True)
dummies_df

,sex_male,embarked_Q,embarked_S
0,1,0,1
1,0,0,0
2,0,0,1
3,0,0,1
4,1,0,1
...,...,...,...
886,1,0,1
887,0,0,1
888,0,0,1
889,1,0,0


In [12]:
# concatenate the dummy var to the original dataframe
df_with_dummies = pd.concat([df, dummies_df], axis=1)
df_with_dummies.columns

Index(['passenger_id', 'survived', 'pclass', 'sex', 'age', 'sibsp', 'parch',
       'fare', 'embarked', 'class', 'deck', 'embark_town', 'alone', 'sex_male',
       'embarked_Q', 'embarked_S'],
      dtype='object')

In [13]:
# verify sex
pd.crosstab(df_with_dummies.sex, df_with_dummies.sex_male)

sex_male,0,1
sex,,
female,314,0
male,0,577


In [14]:
# verify embarked
df_with_dummies.groupby('embarked')[['embarked_Q', 'embarked_S']].sum()

,embarked_Q,embarked_S
embarked,,
C,0.0,0.0
Q,77.0,0.0
S,0.0,644.0


*3. Drop columns: embarked and sex columns (now dummy vars), deck (almost 700 missing values), class (#1 above was a YES). *

In [15]:
# drop columns represented in dummy vars
df_dropped_cols_with_dummies = df_with_dummies.drop(columns=['embarked', 'sex', 'deck', 'class'])

*4. Drop rows where age is missing (177 rows are missing values, and age will likely be a very influential feature).*

In [ ]:
# drop rows where age is missing
df_cleaned = df_dropped_cols_with_dummies[df_dropped_cols_with_dummies.age.notnull()]

In [18]:
# drop 2 rows where embark_town is null. \
df_cleaned = df_cleaned[df_cleaned.embark_town.notnull()]

In [19]:
# take a look at our prepped data
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passenger_id  712 non-null    int64  
 1   survived      712 non-null    int64  
 2   pclass        712 non-null    int64  
 3   age           712 non-null    float64
 4   sibsp         712 non-null    int64  
 5   parch         712 non-null    int64  
 6   fare          712 non-null    float64
 7   embark_town   712 non-null    object 
 8   alone         712 non-null    int64  
 9   sex_male      712 non-null    uint8  
 10  embarked_Q    712 non-null    uint8  
 11  embarked_S    712 non-null    uint8  
dtypes: float64(2), int64(6), object(1), uint8(3)
memory usage: 57.7+ KB


*6. Write prep function*

In [21]:
def clean_titanic(df):
    '''
    This function takes in dataframe as argument, 
    create dummy variables out of sex and embarked, and concats those to original dataframe
    drops columns embarked, sex, deck, class
    drops rows where age (177 rows) or embarked_town (2 rows) are null. 
    it returns the new cleaned dataframe.
    '''
    dummies_df = pd.get_dummies(df[['sex', 'embarked']], drop_first=True)
    df_with_dummies = pd.concat([df, dummies_df], axis=1)
    df_dropped_cols_with_dummies = df_with_dummies.drop(columns=['embarked', 'sex', 'deck', 'class'])
    df_cleaned = df_dropped_cols_with_dummies[df_dropped_cols_with_dummies.age.notnull()]
    return df_cleaned

In [22]:
df = clean_titanic(df)

In [26]:
from sklearn.model_selection import train_test_split
seed = 123
train_and_validate, test = train_test_split(
        df, test_size=0.2, random_state=seed, stratify=df.survived
    )

In [28]:
train, validate = train_test_split(
        train_and_validate,
        test_size=0.3,
        random_state=seed,
        stratify=train_and_validate.survived)

In [30]:
print(train.shape)
print(validate.shape)
print(test.shape)

(399, 12)
(172, 12)
(143, 12)


In [31]:
def train_validate_test_split(prepped_df, seed=123):
    train_and_validate, test = train_test_split(
        prepped_df, test_size=0.2, random_state=seed, stratify=prepped_df.survived
    )
    train, validate = train_test_split(
        train_and_validate,
        test_size=0.3,
        random_state=seed,
        stratify=train_and_validate.survived,
    )
    return train, validate, test

In [32]:
train, validate, test = train_validate_test_split(df)

In [ ]:
print("(Rows, Columns)\nTrain: ", train.shape, 
      "\nValidate: ", validate.shape, 
      "\nTest: ", test.shape)
train.head()

Analyze Survival